### Section 4 - Equilibrating a coarse grained system

When generating the structure of a model, it is often the case that the generated configuration of atoms\
would not readily exist in equilibrium physiological conditions. Individual protein structures may be determined\
using X-ray crystallography, cryo-EM or NMR, and thus the obtained structure data may be that of the crystalline \
or frozen protein or subject to the biases imposed by the experimental technique (e.g. was it resolved in solution\
or possibly on a lipid nano-disk?), if generating structures using Alphafold, protein structure would simply be\
a prediction learned from large databases of these experimental structures. 

If algorithmic procedures such as CHARMM-GUI, COBY, Martinize or some other scripted tool are used to generate\
the initial model this may lead to unnaturally ordered structures that bear little resemblance to their in vivo forms.\
For example; lipids may be packed incorrectly, amino acid fragments may occupy the incorrect rotameric state or proteins\
may need to reorient within a membrane. It is therefore necessary to perform a set of equilibration steps to bring the \
system to a more physiologically accurate equilibrium state before meaningful information may be extracted regarding its\
dynamics. This so-called equilibration phase of MD simulation is split into three distinct stages; energy minimisation, NVT and NPT.

In this part of the session, we will be running a brief energy equilibration and NVT equilibration procedure on our\
 newly generated CG system to illustrate this process.

#### Part 4.1) Checking our initial conditions

First, we will check what files are present in our directory:

In [1]:
%%bash

ls

4-CG_equilibration.ipynb
images
index.ndx
mdps
output_initial_state.pdb
topol.top
toppar


Here we see a selection of files...
1) The initial system configuration generated by our combined martinize & COBY protocol is called "output_initial_state.pdb"\
2) The forcefield and molecule parameter files may be found in toppar\
3) The EM and NVT simulation parameter (.mdp) files can be found in the "mdps" directory


Let's first visualise the initial system... in a separate terminal (NOT this notebook) run the following:

In [2]:
%%bash

vmd output_initial_state.pdb

rlwrap: Command not found.


Info) VMD for LINUXAMD64, version 1.9.2 (December 29, 2014)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 24 CPUs detected.
Info) Free system memory: 53196MB (82%)
Info) Creating CUDA device pool and initializing hardware...
Info) Detected 2 available CUDA accelerators:
Info) [0] NVIDIA GeForce RTX 3060 Ti 38 SM_8.6 @ 1.66 GHz, 7.8GB RAM, KTO, AE2, ZCP
Info) [1] NVIDIA GeForce RTX 3060 Ti 38 SM_8.6 @ 1.66 GHz, 7.8GB RAM, KTO, AE2, ZCP
Info) Detected 2 available TachyonL/OptiX ray tracing accelerators
Warning) Detected X11 'Composite' extension: if incorrect display occurs
Warning) try disabling this

Open the Representations pane via "Graphics > Representations.."

Recreate the following representations:

<img src="images/vmdRepresentation.png" width="350">

Question a) What do you notice about the protein positioning? (Hint: consider the output from the all atom output of the PPM 3.0 server from earlier)

Question b) What do you notice about the lipid positions?

#### Part 4.2) Performing Energy Minimisation

The first equilibration stage is energy minimization (EM).

The purpose of this stage is to ensure that no excessively large forces are present within the system. \
Such forces are usually caused by improper bond geometries and steric clashes between molecules; \
removing these artefacts is the primary goal of the EM phase.

Let's now perform this equilibration phase by running the code in the cell below...

In [3]:
%%bash

# define the local filenames

initial_structure="output_initial_state.pdb"
index_file="index.ndx"
topology_file="topol.top"
minimisation_mdp="mdps/min.mdp"

module load gromacs/2021.4-AVX2-GPU

gmx --version

gmx grompp -f ${minimisation_mdp} -c ${initial_structure} -p ${topology_file} -n ${index_file} -o em.tpr
gmx mdrun -deffnm em -v

                         :-) GROMACS - gmx, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murtol

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Setting the LD random seed to 2096487203

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'molecule_0'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'POPG'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'POPG'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecule type 'CL'

Cleaning up constraints and constant bonded interactions with virtual sites

This run will generate roughly 24 Mb of data


                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [4]:
%%bash

# check that the output files have been generated
ls em.*

em.edr
em.gro
em.log
em.tpr
em.trr


Let's now visualise the trajectory...

In a separate terminal (NOT this notebook), run the following:

In [ ]:
%%bash 

vmd em.trr em.gro

Question c) Which molecules move most over the course of this equilibration phase? Why do you think this is the case?

#### Part 4.3) NVT equilibration

The second equilibration stage is NVT equilibration; the purpose of this phase is to \
impart velocities to the atoms within the static struture generated by EM, raising the \
system temperature to a physiologically relevant equilibrium value.  \
During NVT a temperature coupling regime is introduced into the system’s equations of \
motion. The box dimensions are held constant whilst the atomic velocities are slowly \
increased until the velocity distribution corresponds to an average system temperature \
equal to a user-defined value.  

Typically, any large biomolecules in the system are held in place using position \
restraints during this stage so as to ensure that no undesirable conformational changes \
occur to the molecules whilst the surrounding solvent molecules are allowed to \
correctly orient themselves around the larger structures. 


Let's now run NVT equilibration on the system generated during our previous EM phase..

In [5]:
%%bash

# define the local filenames

initial_structure="em.gro"
index_file="index.ndx"
topology_file="topol.top"
nvt_mdp="mdps/nvt.mdp"

module load gromacs/2021.4-AVX2-GPU

gmx --version

gmx grompp -f ${nvt_mdp} -c ${initial_structure} -p ${topology_file} -n ${index_file} -o nvt.tpr
gmx mdrun -deffnm nvt -v

                         :-) GROMACS - gmx, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murtol

                      :-) GROMACS - gmx grompp, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Setting the LD random seed to 1600691007

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'molecule_0'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'POPG'

Excluding 1 bonded neighbours molecule type 'POPE'

Excluding 1 bonded neighbours molecule type 'POPG'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecule type 'CL'

Setting gen_seed to -1158709249

Velocities were taken from a Maxwell distribution at 323 K

Cleaning up constraints and constant bonded interactions with virtual sites

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 323 K

Calculated rlist for 1x1 atom pair-list as 1.109 nm, buffer size 0.009 nm

Set rlist, assuming 4x4 atom pair-list, to 1.100 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will

                      :-) GROMACS - gmx mdrun, 2021.4 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [6]:
%%bash

# Let's check that the output files have been generated
ls nvt.*

nvt.cpt
nvt.edr
nvt.gro
nvt.log
nvt.tpr
nvt.xtc


If all is well, we should now see files corresponding to the NVT equilibration trajectory,\
a file called "nvt.gro" will be present if the simulation has run to completion.

Let's now visualise the trajectory..

In a separate terminal (NOT this notebook), run the following command and recreate the same \
representations as we used in to visualise the EM trajectory...

In [ ]:
%%bash

vmd nvt.xtc nvt.gro

Question e) How does the motion of atoms in the NVT phase differ to the EM phase? Why do you think this is?

After equilibration the system should look something like the following image (your representations in the above phases will differ)

<img src="images/nvt.png" width="750">

#### Part 4.4) NPT Equilibration

The final equilibration phase is NPT equilibration; the purpose of this phase is to take\
the system generated during NVT equilibration and raise or lower its pressure to physiological\
levels (i.e. 1 bar). During NPT, both a temperature and pressure coupling regime are applied\
to the system’s equations of motion. As such, the previously equilibrated temperature is held\
constant while the box dimensions and particle positions are scaled until the system reaches\
a roughly constant average pressure.

This is arguably the most important and nuanced stage in the equilibration process; it is \
often the case with more complex systems that numerous NPT stages are required with varied \
position restraints in order to allow all of the large biomolecules in the system to relax into\
their equilibrium conformations without causing non-physical deformations to the other structures\
in the system. This is especially the case when dealing with large structures that directly\
interact with many components of the system. 

Once the NPT phase is complete, the equilibrated system is ready for unrestrained production MD, \
the stage in which meaningful dynamical data may be extracted from the system. Even so, when \
working with membranes, it is wise to ignore <i> at least </i> the first 100 ns when performing any \
analysis of membrane properties. This is required because the high density of lipids within the \
bilayer often take a considerable amount of time to find the correct organisation and orientation \
with respect to each other and any external structures embedded within the membrane.

The Supplementary Information for the following paper has a lot of data on the time required for\
several membrane properties to equilibrate: 

<b>CHARMM-GUI Supports Hydrogen Mass Repartitioning and Different Protonation States of Phosphates in Lipopolysaccharides</b>\
<i>Gao Y., Lee J., Smith I.P.S., Lee H., Kim S., Qi Y., Klauda J.B., Widmalm G., Khalid S., Im W.</i>\
https://pubs.acs.org/doi/10.1021/acs.jcim.0c01360 